In [66]:
import numpy as np
import pandas as pd
import os
import re

In [67]:
"""
This script calculates average path-lengths in the X and Y directions (<Lx> and <Ly>) in quark-gluon plasma.
Input: quark-gluon plasma temperature profile, given as a matrix with the following columns: (tau, X, Y, T)
"""

crit_temp = 0.16

# loop over centrality classes
for cen in ['10.20', '20.30', '30.40', '40.50']:
    
    path = '.'
    
    # Look up the correct file
    for file in os.listdir(path):
        
        if re.search(r"temp_evo.*" + cen + ".dat" , file):
                filename = path + '/' + file
    
    # Import QGP temperature profile
    # and convert it to a numpy array
    # (using Pandas to import = faster)
    profile = pd.read_csv(filename, header = None, sep = '\t').to_numpy()
    
    # Find length of one snapshot
    snapshot_size = max(count for count, elem in enumerate(profile[:, 0]) if elem == profile[0, 0])
    
    # Find the number of distinct snapshots in the temperature profile
    num_of_tsteps = len(set(profile[:, 0]))
    
    # Create empty numpy arrays which will store
    # average paths in x and y directions
    avg_paths_x = np.empty(shape = (0, 2))
    avg_paths_y = np.empty(shape = (0, 2))

    for t in range(0, num_of_tsteps + 1):

        # calculate <Ly>
        for row in profile[t * (snapshot_size + 1) : (t + 1) * (snapshot_size + 1)]:
        
            # look only at the first quadrant
            if (row[1] < 0.0) or (row[2] < 0.0):
                continue
            
            # break if temperature at (0, 0) is under critical
            elif (row[1] == 0.0) and (row[2] == 0.0) and (row[3] < crit_temp):
                break
        
            elif row[1] == 0.0:
            
                if row[3] >= crit_temp:
                    continue
            
                elif row[3] < crit_temp:
                    avg_paths_y = np.append(avg_paths_y, np.array([[row[0], row[2]]]), axis = 0)
                    break
                
        # calculate <Lx>
        for row in profile[t * (snapshot_size + 1) : (t + 1) * (snapshot_size + 1)]:
            
            if (row[1] < 0.0) or (row[2] < 0.0):
                continue
            
            # break if temperature at (0, 0) is under critical
            elif (row[1] == 0.0) and (row[2] == 0.0) and (row[3] < crit_temp):
                break
            
            # we want paths along the X axis here
            elif (row[1] > 0.0) and (row[3] >= crit_temp):
                continue
            
            # look at paths along x axis (y = 0)
            elif ((row[1] > 0.0) and (row[2] == 0.0) and (row[3] < crit_temp)):
                avg_paths_x = np.append(avg_paths_x, np.array([[row[0], row[1]]]), axis = 0)
                break
    
    # combine the arrays containing Lx and Ly data
    avg_paths = np.append(avg_paths_x, avg_paths_y, axis = 1)
    
    # erase duplicated tau column
    avg_paths = np.delete(avg_paths, 2, axis = 1)
    
    # calculate anisotropies as (Ly - Lx)/(Ly + Lx)
    anisotropies = np.divide(avg_paths[:, 2] - avg_paths[:, 1], avg_paths[:, 2] + avg_paths[:, 1])
    
    # append anisotropies list to avg_paths
    avg_paths = np.hstack((avg_paths, anisotropies[:, None]))
    
    # Export data to csv file
    pd.DataFrame(avg_paths).to_csv("MUSIC_paths_" + cen + ".csv", index = False, header = ["tau", "Lx", "Ly", "Anisotropy"])